In [10]:
import torch
print(torch.cuda.is_available())

True


In [11]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device detected")

Torch version: 2.2.0+cu121
CUDA available: True
CUDA device name: NVIDIA GeForce RTX 4070 SUPER


In [12]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.2.0+cu121
CUDA available: True


In [13]:
from torchtext.datasets import UDPOS, CoNLL2000Chunking

In [14]:
# Load the train, validation, and test datasets
train_iter, valid_iter, test_iter = UDPOS(root='.data', split=('train', 'valid', 'test'))

In [15]:
# Look at the first few items in the training set
for i, item in enumerate(train_iter):
    print(item)
    if i == 0:  # Print first 5 items
        break

[['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.'], ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']]


In [21]:
# Load the train, validation, and test datasets
train_iter, test_iter = CoNLL2000Chunking(root='.data', split=('train', 'test'))

In [22]:
# Look at the first few items in the training set
for i, item in enumerate(train_iter):
    print(item)
    if i == 0:  # Print first 5 items
        break

[['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.'], ['NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.'], ['B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O']]


In [16]:
train_iter

ShardingFilterIterDataPipe

In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

pipeline = TokenClassificationPipeline(model=model, tokenizer=tokenizer)
outputs = pipeline("A test example")
print(outputs)


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 46


In [ ]:
label_map = model.config.id2label

In [ ]:
for id, label in label_map.items():
    print(f"ID: {id}, Tag: {label}")

ID: 0, Tag: O
ID: 1, Tag: ``
ID: 2, Tag: ,
ID: 3, Tag: :
ID: 4, Tag: .
ID: 5, Tag: ''
ID: 6, Tag: $
ID: 7, Tag: #
ID: 8, Tag: CC
ID: 9, Tag: CD
ID: 10, Tag: DT
ID: 11, Tag: EX
ID: 12, Tag: FW
ID: 13, Tag: IN
ID: 14, Tag: JJ
ID: 15, Tag: JJR
ID: 16, Tag: JJS
ID: 17, Tag: -LRB-
ID: 18, Tag: LS
ID: 19, Tag: MD
ID: 20, Tag: NN
ID: 21, Tag: NNP
ID: 22, Tag: NNPS
ID: 23, Tag: NNS
ID: 24, Tag: PDT
ID: 25, Tag: POS
ID: 26, Tag: PRP
ID: 27, Tag: PRP$
ID: 28, Tag: RB
ID: 29, Tag: RBR
ID: 30, Tag: RBS
ID: 31, Tag: RP
ID: 32, Tag: -RRB-
ID: 33, Tag: SYM
ID: 34, Tag: TO
ID: 35, Tag: UH
ID: 36, Tag: VB
ID: 37, Tag: VBD
ID: 38, Tag: VBG
ID: 39, Tag: VBN
ID: 40, Tag: VBP
ID: 41, Tag: VBZ
ID: 42, Tag: WDT
ID: 43, Tag: WP
ID: 44, Tag: WP$
ID: 45, Tag: WRB


In [ ]:
# Load the train, validation, and test datasets
train_iter, test_iter = CoNLL2000Chunking(root='.data', split=('train', 'test'))

In [18]:
# Function to extract unique chunk tags
def extract_tags(data_iter):
    tags = set()
    for sentence in data_iter:
        for word, pos, chunk in sentence:
            tags.add(chunk)
    return tags

# Extract unique chunk tags from the training data
train_tags = extract_tags(train_iter)
print("Unique Chunk Tags:", train_tags)

/opt/conda/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


ValueError: too many values to unpack (expected 3)